<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width = 400, align = "center"></a>

<h1 align=center><font size = 5>COLLABORATIVE FILTERING</font></h1>

Los sistemas de recomendación son una colección de algoritmos utilizados para recomendar elementos a los usuarios en función de la información tomada del usuario. Estos sistemas se han vuelto omnipresentes y se pueden ver comúnmente en tiendas en línea, bases de datos de películas y buscadores de trabajo. En este cuaderno, exploraremos sistemas de recomendación basados en el filtrado colaborativo e implementaremos una versión simple de uno usando Python y la biblioteca Pandas.

### Table of contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
- <p><a href="#ref1">Acquiring the Data</a></p>
- <p><a href="#ref2">Preprocessing</a></p>
- <p><a href="#ref3">Collaborative Filtering</a></p>
<p></p>
</div>
<br>

<hr>
<a id="ref1"></a>
# Acquiring the Data

Para adquirir y extraer los datos, simplemente ejecute los siguientes scripts de Bash:
Conjunto de datos adquirido de[GroupLens](http://grouplens.org/datasets/movielens/). Vamos a descargar el conjunto de datos. Para descargar los datos, usaremos **`!wget`**. Para descargar los datos, usaremos `!wget` para descargarlo de IBM Object Storage.
__Did you know?__ When it comes to Machine Learning, you will likely be working with large datasets. As a business, where can you host your data? IBM is offering a unique opportunity for businesses, with 10 Tb of IBM Cloud Object Storage: [Sign up now for free](http://cocl.us/ML0101EN-IBM-Offer-CC)

In [ ]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

¡Ahora estás listo para comenzar a trabajar con los datos!

<hr>
<a id="ref2"></a>
# Preprocessing

Primero, eliminemos todas las importaciones:

In [ ]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Ahora leamos cada archivo en su Dataframes:

In [ ]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('movies.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('ratings.csv')

También echemos un vistazo a cómo están organizadas cada una de ellas:

In [ ]:
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df.head()

Por lo tanto, cada película tiene una identificación única, un título con su año de lanzamiento junto con él (que puede contener caracteres Unicode) y varios géneros diferentes en el mismo campo. Eliminemos el año de la columna del título y colóquelo en el suyo usando el práctico [extract](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.extract.html#pandas.Series.str.extract) función que tiene Pandas.

Quitemos el año del __title__ columna mediante la función de reemplazo de pandas y almacenar en una nueva columna  __year__ 

In [ ]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

¡Veamos el resultado!

In [ ]:
movies_df.head()

Con eso, eliminemos también la columna de géneros, ya que no la necesitaremos para este sistema de recomendación en particular.

In [ ]:
#Dropping the genres column
movies_df = movies_df.drop('genres', 1)

Aquí está el  dataframe de películas final

In [ ]:
movies_df.head()

<br>

A continuación, veamos las calificaciones de dataframe.

In [ ]:
ratings_df.head()

Cada fila en el marco de datos de calificaciones tiene una identificación de usuario asociada con al menos una película, una calificación y una marca de tiempo que muestra cuándo la revisaron. No necesitaremos la columna de marca de tiempo, así que eliminémosla para guardarla en la memoria.

In [ ]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)

Así es como se ve el Dataframe de calificaciones finales:

In [ ]:
ratings_df.head()

<hr>
<a id="ref3"></a>
# Collaborative Filtering // Filtración colaborativa

Ahora, es hora de comenzar nuestro trabajo en sistemas de recomendación.

La primera técnica que vamos a echar un vistazo se llama __Collaborative Filtering__, que también se conoce como __User-User Filtering__. Como se insinúa por su nombre alternativo, esta técnica utiliza otros usuarios para recomendar elementos al usuario de entrada. Intenta encontrar usuarios que tengan preferencias y opiniones similares a la entrada y luego recomienda elementos que les han gustado a la entrada. Existen varios métodos para encontrar usuarios similares (incluso algunos que utilizan Machine Learning), y el que usaremos aquí se basará en __Pearson Correlation Function__.

<img src="https://ibm.box.com/shared/static/1ql8cbwhtkmbr6nge5e706ikzm5mua5w.png" width=800px>

El proceso para crear un sistema de recomendación basado en el usuario es el siguiente:
- Select a user with the movies the user has watched
- Based on his rating to movies, find the top X neighbours 
- Get the watched movie record of the user for each neighbour.
- Calculate a similarity score using some formula
- Recommend the items with the highest score


Comencemos creando un usuario de entrada para recomendar películas para:

Aviso: para agregar más películas, simplemente aumente la cantidad de elementos en la entrada de usuario. ¡Siéntase libre de agregar más! Solo asegúrese de escribirlo con letras mayúsculas y si una película comienza con un "The", like "The Matrix" then write it in like this: 'Matrix, The' .

In [ ]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

#### Add movieId to input user // Añadir movieId al usuario de entrada
Una vez completada la entrada, extraigamos los ID de las películas de entrada del marco de datos de las películas y añádalas.

Podemos lograr esto filtrando primero las filas que contienen el título de las películas de entrada y luego fusionando este subconjunto con el marco de datos de entrada. También soltamos columnas innecesarias para la entrada para ahorrar espacio en la memoria.

In [ ]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

#### The users who has seen the same movies
Ahora con los ID de películas en nuestra entrada, ahora podemos obtener el subconjunto de usuarios que han visto y revisado las películas en nuestra entrada.


In [ ]:
#Filtering out users that have watched movies that the input has watched and storing it
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

Ahora agrupamos las filas por ID de usuario.

In [ ]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])

echemos un vistazo a uno de los usuarios, p. el que tieneuserID=1130

In [ ]:
userSubsetGroup.get_group(1130)

También clasifiquemos estos grupos para que los usuarios que comparten más películas en común con la entrada tengan mayor prioridad. Esto proporciona una recomendación más rica, ya que no revisaremos a todos los usuarios.

In [ ]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

Ahora veamos a la primera usuaria

In [ ]:
userSubsetGroup[0:3]

#### Similarity of users to input user
A continuación, vamos a comparar a todos los usuarios (¡no a todos!) Con nuestro usuario especificado y encontrar el que sea más similar.
vamos a descubrir qué tan similar es cada usuario a la entrada a través del __Pearson Correlation Coefficient__. Se utiliza para medir la fuerza de una asociación lineal entre dos variables. La fórmula para encontrar este coeficiente entre los conjuntos X e Y con N valores se puede ver en la imagen a continuación.

¿Por qué correlación de Pearson (**Pearson Correlation**)?

La correlación de Pearson es invariante con la escala, es decir, multiplicar todos los elementos por una constante distinta de cero o agregar cualquier constante a todos los elementos. Por ejemplo, si tiene dos vectores X e Y, entonces, pearson (X, Y) == pearson (X, 2 * Y + 3). Esta es una propiedad bastante importante en los sistemas de recomendación porque, por ejemplo, dos usuarios podrían calificar dos series de elementos totalmente diferentes en términos de tasas absolutas, pero serían usuarios similares (es decir, con ideas similares) con tasas similares en varias escalas.

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

Los valores dados por la fórmula varían de r = -1 a r = 1, donde 1 forma una correlación directa entre las dos entidades (significa una correlación positiva perfecta) y -1 forma una correlación negativa perfecta.

En nuestro caso, un 1 significa que los dos usuarios tienen gustos similares, mientras que un -1 significa lo contrario.

Seleccionaremos un subconjunto de usuarios para iterar. Este límite se impone porque no queremos perder mucho tiempo pasando por cada usuario.

In [ ]:
userSubsetGroup = userSubsetGroup[0:100]

Ahora, calculamos la correlación de Pearson entre el usuario de entrada y el grupo de subconjuntos, y la almacenamos en un diccionario, donde la clave es el ID del usuario y el valor es el coeficiente


In [ ]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [ ]:
pearsonCorrelationDict.items()

In [ ]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

#### The top x similar users to input user // Los mejores x usuarios similares al usuario de entrada
Ahora obtengamos los 50 principales usuarios que son más similares a la entrada.

In [ ]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

Ahora, comencemos a recomendar películas al usuario de entrada.

#### Rating of selected users to all movies
Vamos a hacer esto tomando el promedio ponderado de las clasificaciones de las películas usando la Correlación de Pearson como el peso. Pero para hacer esto, primero necesitamos que los usuarios vean las películas en nuestro __pearsonDF__ del marco de datos de calificaciones y luego almacenar su correlación en una nueva columna llamada _similarityIndex ". Esto se logra a continuación mediante la fusión de estas dos tablas.

In [ ]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

Ahora todo lo que tenemos que hacer es simplemente multiplicar la calificación de la película por su peso (El índice de similitud), luego resumir las nuevas calificaciones y dividirlo por la suma de los pesos.

Podemos hacer esto fácilmente simplemente multiplicando dos columnas, luego agrupando el marco de datos por movieId y luego dividiendo dos columnas:

Muestra la idea de todos los usuarios similares a las películas candidatas para el usuario de entrada:

In [ ]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

In [ ]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

In [ ]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

¡Ahora ordenemos y veamos las 20 mejores películas recomendadas por el algoritmo!

In [ ]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

In [ ]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

### Ventajas y desventajas del filtrado colaborativo **Collaborative Filtering**

##### Advantages
* Toma en consideración las calificaciones de otros usuarios
* No necesita estudiar o extraer información del artículo recomendado
* Se adapta a los intereses del usuario que pueden cambiar con el tiempo.

##### Disadvantages
* La función de aproximación puede ser lenta
* Puede haber una cantidad baja de usuarios para aproximar
* Problemas de privacidad al intentar conocer las preferencias del usuario

## Want to learn more?

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: [SPSS Modeler](http://cocl.us/ML0101EN-SPSSModeler).

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at [Watson Studio](https://cocl.us/ML0101EN_DSX)

### Thanks for completing this lesson!

Notebook created by: <a href = "https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, Gabriel Garcez Barros Sousa


<hr>
Copyright &copy; 2018 [Cognitive Class](https://cocl.us/DX0108EN_CC). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).​